In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pickle

# from ema_workbench import (
#     Model,
#     Policy,
#     ema_logging,
#     SequentialEvaluator,
#     MultiprocessingEvaluator, 
#     perform_experiments,
#     Samplers, 
# )
# from SALib.analyze import sobol
# from ema_workbench.em_framework.salib_samplers import get_SALib_problem
# from ema_workbench.analysis import (feature_scoring, pairs_plotting)
from ema_workbench.analysis import prim
from ema_workbench import ema_logging
from ema_workbench import SequentialEvaluator

# from dike_model_function import DikeNetwork  # @UnresolvedImport
from problem_formulation import get_model_for_problem_formulation, sum_over, sum_over_time
# from ema_workbench import Model, RealParameter, TimeSeriesOutcome, ScalarOutcome



In [4]:
# First, get the model
dike_model, planning_steps = get_model_for_problem_formulation(3)
print(dike_model)
print(planning_steps)

# We will need experiments and outcomes, for the sake of scoping they are listed here
experiments, outcomes = ...

[0 1 2]


TypeError: cannot unpack non-iterable ellipsis object

In [ ]:
# Optional: run the model a bunch of times and save it to a file, change boolean below if you want to run it or not
# False means: don't run it and use the presaved data
# True means: do run the program
want_to_run = False

if want_to_run:
    with SequentialEvaluator(dike_model) as evaluator:
        results = evaluator.perform_experiments(scenarios=100, policies=30)
        
    # split the results into experiments and outcomes
    experiments, outcomes = results
    experiments.info()
    
    #save the results so that this (which takes like 2 hours) doesn't need to be rerun over and over
    with open('data from all the runs', 'wb') as file:
        pickle.dump(results, file)
    

In [ ]:
if not want_to_run:
    # open the results so that you do prim on them
    with open('data from all the runs', 'rb') as file:
        loaded_data = pickle.load(file)
    experiments, outcomes = loaded_data
    print(outcomes)

In [ ]:
# Datacleaning
experiments.head()

# Remove irrelevant columns
experiments.drop(columns=['policy', 'scenario', 'model'], inplace = True)

In [ ]:
df = pd.DataFrame(outcomes)

# Summing the total costs
df['Total Costs'] = df[['A.1 Total Costs', 'A.2 Total Costs', 'A.3 Total Costs', 
                        'A.4 Total Costs', 'A.5 Total Costs', 'RfR Total Costs', 
                        'Expected Evacuation Costs']].sum(axis=1)

# Summing the total expected number of deaths
df['Total Deaths'] = df[['A.1_Expected Number of Deaths', 'A.2_Expected Number of Deaths', 
                         'A.3_Expected Number of Deaths', 'A.4_Expected Number of Deaths', 
                         'A.5_Expected Number of Deaths']].sum(axis=1)


In [ ]:
#250 000 000 budget euro

# Define what policies are deemed unacceptable
df['Unacceptable'] = ( (df['Total Deaths'] > 0) & (df['Total Costs'] > 300000000)).astype(int) 
# Turn these values into the y variable in the form of an array
y = df['Unacceptable'].values
print(y)
print(df['Unacceptable'].value_counts(1))

In [ ]:
ema_logging.log_to_stderr(ema_logging.INFO)

#Perform prim
prim_alg = prim.Prim(experiments, y, threshold=0.7, peel_alpha=0.05)
box1 = prim_alg.find_box()
box1.show_tradeoff(annotated=True)
plt.show()



In [ ]:
# Pick a point
point = 5
box1.inspect(point)

In [ ]:
# Make the weird plot thing
box1.inspect(point, style='graph')
plt.show()

In [ ]:
# See how reproducable it is
box1.resample(point)

In [ ]:
#Make pretty plots for the chosen point
box1.select(point)
box1.show_pairs_scatter()
fig = plt.gcf()
fig.set_size_inches(12,12)
plt.show()